In [1]:
import ueye
import numpy as np
from matplotlib.pyplot import *
import os
from skimage.measure import label
from sklearn.metrics import cohen_kappa_score as cohenskappa

# EM algorithms 
adapted from Engbert et al.

**Parameters to be tuned:**

- lambda



In [16]:
# Cross Validation: take the cross validation set, split into 10folds. Use 90% for parameter tuning and 10 % to test.
lambdas = np.arange(1,11,.2) #lambda candidates


K = 10 #10fold cross validation
Ntrain = [950,950,330]
datasets = ['dataset1','dataset2','dataset3']
Sampfreq=[1000,1000,500]
min_sacc_dur = 6
min_distance = [10,5,10]
for i,dset in enumerate(datasets):
    
    ntrain = Ntrain[i]
    min_sacc_dist = min_distance[i]
    sampfreq = Sampfreq[i]
    fac = 1000/sampfreq
    datapath = '../data/' + dset
    print(datapath)
    X = np.loadtxt(datapath+'/'+dset+'_'+str(sampfreq)+'hz_X_train.csv',delimiter=',')
    Y = np.loadtxt(datapath+'/'+dset+'_'+str(sampfreq)+'hz_Y_train.csv',delimiter=',')
    L = np.loadtxt(datapath+'/'+dset+'_'+str(sampfreq)+'hz_Labels_train.csv',delimiter=',')
    L = (L==1).astype(float)
    # chose same subset as was used for network
    np.random.seed(1)
    randind = np.random.permutation(X.shape[0])
    X = X[randind[:ntrain],:]
    Y = Y[randind[:ntrain],:]
    L = L[randind[:ntrain],:]

    n_samples,n_time = X.shape
    
    np.random.seed(1)
    n_test = int(n_samples/K)
    Kappa = np.zeros(K)
    F1 = np.zeros(K)
    Kappa_lam_F1 = np.zeros(K)
    F1_lam_F1 = np.zeros(K)
    On = []
    Off = []
    On_lam_F1 = []
    Off_lam_F1 = []
    indices = np.random.permutation(n_samples) #indices from training and test set
    # CROSS VALIDATION
    for i in range(K):
        ind_train = indices.copy()
        if i==K-1:
            ind_train = np.array(np.delete(ind_train,range(n_test*i,n_samples)))
            ind_test = indices[n_test*i:]
        else:
            ind_train = np.array(np.delete(ind_train,range(n_test*i,n_test*(i+1))))
            ind_test = indices[n_test*i:n_test*(i+1)]
            
        # training and test set
        Xtrain = X[ind_train,:].copy()
        Ytrain = Y[ind_train,:].copy()
        Xtest = X[ind_test,:].copy()
        Ytest = Y[ind_test,:].copy()
        Ltrain = L[ind_train,:].copy()  
        Ltest = L[ind_test,:].copy()
        kappa = np.zeros(len(lambdas))
        f1 = np.zeros(len(lambdas))
        for j,lam in enumerate(lambdas):
            Sacc_out = ueye.functions.EM_saccade_detection(Xtrain,Ytrain,lambda_param=lam,
                                                          min_sacc_dist=min_sacc_dist,min_sacc_dur=min_sacc_dur,
                                                           sampfreq=sampfreq)
            kappa[j] = cohenskappa(Ltrain.astype(float).flatten(),Sacc_out.astype(float).flatten())
            true_pos,false_pos,false_neg,on_distance,off_distance = ueye.functions.accuracy(Ltrain.astype(float),Sacc_out.astype(float))    
            f1[j] = (2 * true_pos)/(2 * true_pos + false_neg + false_pos) 
        best_lam = lambdas[np.argmax(kappa)]
        best_lam_F1 = lambdas[np.argmax(f1)]
        print('best lambda for cohen''s kappa:',best_lam)
        print('best lambda for F1:',best_lam_F1)
        # TEST
        # with lambda optimized for Cohen's kappa
        Sacc_out = ueye.functions.EM_saccade_detection(Xtest,Ytest,lambda_param=best_lam,
                                                      min_sacc_dist=min_sacc_dist,min_sacc_dur=min_sacc_dur,
                                                           sampfreq=sampfreq)
    
        Kappa[i] = cohenskappa(Ltest.astype(float).flatten(),Sacc_out.astype(float).flatten()) 
        true_pos,false_pos,false_neg,on_distance,off_distance = ueye.functions.accuracy(Sacc_out.astype(float),Ltest.astype(float))
        F1[i] = (2 * true_pos)/(2 * true_pos + false_neg + false_pos)
        
        On.append(on_distance)
        Off.append(off_distance)
        print('Kappa, opt. Kappa:',Kappa[i])
        print('Average onset distance, opt. Kappa:',np.mean(np.abs(on_distance)))
        print('Average offset distance, opt. Kappa:',np.mean(np.abs(off_distance)))
        
        # with lambda optimized for F1
        Sacc_out = ueye.functions.EM_saccade_detection(Xtest,Ytest,lambda_param=best_lam_F1,
                                                      min_sacc_dist=min_sacc_dist,min_sacc_dur=min_sacc_dur,
                                                           sampfreq=sampfreq)
    
        Kappa_lam_F1[i] = cohenskappa(Ltest.astype(float).flatten(),Sacc_out.astype(float).flatten()) 
        true_pos,false_pos,false_neg,on_distance,off_distance = ueye.functions.accuracy(Sacc_out.astype(float),Ltest.astype(float))
        F1_lam_F1[i] = (2 * true_pos)/(2 * true_pos + false_neg + false_pos)  
        print('Kappa, opt. F1:',Kappa_lam_F1[i])
        On_lam_F1.append(on_distance)
        Off_lam_F1.append(off_distance)
        print('Average onset distance, opt. F1:',np.mean(np.abs(on_distance)))
        print('Average offset distance, opt. F1:',np.mean(np.abs(off_distance)))
    Performance = {
        'kappa': Kappa,
        'f1': F1,
        'kappa_lam_f1':Kappa_lam_F1,
        'F1_lam_f1':F1_lam_F1,
        'on': On,
        'off': Off,
        'on_lam_f1':On_lam_F1,
        'off_lam_f1':Off_lam_F1
        }
    np.save('summary/crossvalidation/em_'+dset,Performance)
        


../data/dataset1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/Users/mebellet/Dropbox/ueye/analysis scripts/ueye/functions.py:465: RuntimeWarning: invalid value encountered in greater
  Sacc_out[i,:]=(np.divide(Vx[i,:],(lambda_param*np.sqrt(Eta_x[i])))**2+np.divide(Vy[i,:],(lambda_param*np.sqrt(Eta_y[i])))**2)>1


best lambda for cohens kappa: 4.3999999999999995
best lambda for F1: 5.399999999999999
Kappa, opt. Kappa: 0.6454889756866276
Average onset distance, opt. Kappa: 6.469387755102041
Average offset distance, opt. Kappa: 11.993197278911564
Kappa, opt. F1: 0.6018546312906181
Average onset distance, opt. F1: 6.6879432624113475
Average offset distance, opt. F1: 14.51063829787234
best lambda for cohens kappa: 4.3999999999999995
best lambda for F1: 5.799999999999999
Kappa, opt. Kappa: 0.6617431166934404
Average onset distance, opt. Kappa: 4.92
Average offset distance, opt. Kappa: 12.24
Kappa, opt. F1: 0.6006338617942155
Average onset distance, opt. F1: 5.485714285714286
Average offset distance, opt. F1: 14.9
best lambda for cohens kappa: 4.3999999999999995
best lambda for F1: 5.399999999999999
Kappa, opt. Kappa: 0.6687613814281124
Average onset distance, opt. Kappa: 4.985915492957746
Average offset distance, opt. Kappa: 11.253521126760564
Kappa, opt. F1: 0.6308572082456133
Average onset distance

best lambda for cohens kappa: 3.999999999999999
best lambda for F1: 4.6
Kappa, opt. Kappa: 0.5809624066213888
Average onset distance, opt. Kappa: 1.4675324675324675
Average offset distance, opt. Kappa: 3.4935064935064934
Kappa, opt. F1: 0.5826762231540299
Average onset distance, opt. F1: 1.408450704225352
Average offset distance, opt. F1: 3.732394366197183
best lambda for cohens kappa: 3.999999999999999
best lambda for F1: 4.6
Kappa, opt. Kappa: 0.5286961020224891
Average onset distance, opt. Kappa: 1.9482758620689655
Average offset distance, opt. Kappa: 4.0344827586206895
Kappa, opt. F1: 0.4777778690700045
Average onset distance, opt. F1: 1.607843137254902
Average offset distance, opt. F1: 4.176470588235294
best lambda for cohens kappa: 3.999999999999999
best lambda for F1: 4.6
Kappa, opt. Kappa: 0.5840221031188482
Average onset distance, opt. Kappa: 1.5507246376811594
Average offset distance, opt. Kappa: 3.36231884057971
Kappa, opt. F1: 0.531599132243008
Average onset distance, opt. 